### Access MIP and EDA
***

Import libraries:

In [38]:
import pandas as pd
import pyodbc as p

In [39]:
def limit10_fn(conn,db_name, table_name):
    query="""select * from {db_name}.{table_name} limit 10""".format(db_name=db_name,table_name=table_name)
    return pd.read_sql(query,conn)

In [40]:
sql_query="""select * from 
(
with cte as (
select distinct
ben.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num 
,ben.dep_fk 
,ben.comp_name
,mipst_dbo.tsf_ccm_template.template_label	
,mipst_dbo.tsf_ccm_template.template_description
,user_code
,replace(substring(call_start_datetime,1,10), '-','') as call_date 
,coalesce(g.comp_name,'none') as group_name	
,dep_type
,ben.comp_fk
,scheme_fk

FROM mipst_dbo.tsf_ccm_template
left join mipst_dbo.tsf_cct_call call ON call.template_obj = mipst_dbo.tsf_ccm_template.template_obj and call.status_key = 'cc_StCctcaComp' 
inner join mipst_dbo.tsd_cc_relationship real ON real.owning_obj::NUMERIC = call.call_obj
and UPPER(owning_entity_mnemonic) = 'CCTCA'
AND   UPPER(related_entity_mnemonic) in ('MEMBER','MEMDEP')
inner join  mipbi_dbo.td_beneficiary ben ON ben.ms_pk = coalesce(nullif(split_part(related_key, '|',1), ''), '0') --and scheme_fk='126'
and dep_fk::int	=coalesce(nullif(split_part(related_key, '|',2), ''), '0')::int
left join mipst_dbo.tsd_ben_memcom g  on  (case when ben.group_code='0' then ben.comp_fk else ben.group_code end)=g.comp_fk
) select cte.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num as identity_number 
,dep_fk as dependent_no
,dep_type
,cte.comp_name as companyname
, group_name
,cte.template_label	
,cte.template_description
,cte.user_code
, cte.call_date::varchar::date as call_date 
, array_to_string(array_agg(distinct (coalesce((c.Problem||'-'||c.sub_category),'') ||','||coalesce(l.Problem ,'')||','||coalesce(f.Problem ,''))),',') as Problem_name
from cte 
INNER JOIN 
mipst_dbo.tsd_dates on mipst_dbo.tsd_dates.date_pk::VARCHAR =call_date 
left join 
mipst_dbo.legal_raw_2021 l on cte.ms_pk=l.ms_pk and cte.dep_fk=l.dependent_no::int and  cte.call_date::varchar::date =l.call_date and cte.template_label=l.template_label and cte.comp_name=l.comp_name 
				  and  cte.scheme_fk=l.scheme_fk and cte.user_code=l.user_code  left join
mipst_dbo.councelling_raw_2021 c on cte.ms_pk=c.ms_pk and cte.dep_fk=c.dependent_no::int and  cte.call_date::varchar::date =c.call_date and cte.template_label=c.template_label and cte.comp_name=c.comp_name 			 
and  cte.scheme_fk=c.scheme_fk and cte.user_code=c.user_code  left join
mipst_dbo.financial_raw_2021 f on cte.ms_pk=f.ms_pk and cte.dep_fk=f.dependent_no::int and  cte.call_date::varchar::date =f.call_date and cte.template_label=f.template_label and cte.comp_name=f.comp_name 			 
and  cte.scheme_fk=f.scheme_fk and cte.user_code=f.user_code  
group by cte.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num  
,dep_fk
,dep_type				  
,cte.comp_name 
, group_name
,cte.template_label	
,cte.template_description
,cte.user_code
, cte.call_date::varchar::date 
) a"""

Here we connect to the __kx_bi__ postgres database and execute the above query to generate the EAP raw data set:

In [41]:
conn_str = (
    "DRIVER={PostgreSQL ODBC Driver(UNICODE)};"
    "DATABASE=kx_bi;"
    "UID=postgres;"
    "PWD=postgres;"
    "SERVER=172.31.244.44;"
    "PORT=5432;"
    )

conn = p.connect(conn_str)
# cursor = conn.cursor()

eapraw_df = pd.read_sql(sql_query,conn)

In [49]:
eapraw_df.iloc[3366]["problem_name"]
# [eapraw_df.ms_pk=="""01678068"""]

',,Debt Management,,,WillsandEstate,,,Willsdrafting'

In [50]:
eapraw_df.iloc[3366]["problem_name"]

',,Debt Management,,,WillsandEstate,,,Willsdrafting'

In [42]:
list(eapraw_df)

['ms_pk',
 'scheme_name',
 'first_name',
 'surname',
 'gender',
 'birth_date',
 'main_email',
 'main_work_tel',
 'identity_number',
 'dependent_no',
 'dep_type',
 'companyname',
 'group_name',
 'template_label',
 'template_description',
 'user_code',
 'call_date',
 'problem_name']

In [ ]:
"""
mipst_dbo.tsf_ccm_template
mipst_dbo.tsf_cct_call
mipst_dbo.tsd_cc_relationship
mipbi_dbo.td_beneficiary
mipst_dbo.tsd_ben_memcom
mipst_dbo.tsd_dates
mipst_dbo.legal_raw_2021
mipst_dbo.councelling_raw_2021
mipst_dbo.financial_raw_2021
"""

In [15]:
tsf_ccm_template_df=limit10_fn(conn=conn,db_name="mipst_dbo", table_name="tsf_ccm_template")
tsf_cct_call_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="tsf_cct_call")
tsd_cc_relationship_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="tsd_cc_relationship")
td_beneficiary_df=limit10_fn(conn=conn,db_name="mipbi_dbo",table_name="td_beneficiary")
tsd_ben_memcom_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="tsd_ben_memcom")
tsd_dates_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="tsd_dates")
legal_raw_2021_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="legal_raw_2021")
councelling_raw_2021_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="councelling_raw_2021")
financial_raw_2021_df=limit10_fn(conn=conn,db_name="mipst_dbo",table_name="financial_raw_2021")

In [20]:
# tsf_ccm_template_df
list(tsf_ccm_template_df)

['template_obj',
 'status_key',
 'category_key',
 'object_obj',
 'template_code',
 'template_label',
 'template_description',
 'template_field_namelist',
 'template_field_attriblist',
 'last_modified_datetime']

In [21]:
list(tsf_cct_call_df)

['call_obj',
 'template_obj',
 'status_key',
 'call_reference',
 'user_code',
 'direction_acronym_key',
 'method_acronym_key',
 'abort_reason_acronym_key',
 'call_field_namelist',
 'call_field_attriblist',
 'call_caller_name',
 'call_telephone_number',
 'call_start_datetime',
 'call_end_datetime',
 'call_enquiry',
 'call_response',
 'call_wordindex',
 'last_modified_datetime',
 'call_start_datetime_int']

In [22]:
list(tsd_cc_relationship_df)

['relationship_obj',
 'relation_key',
 'owning_entity_mnemonic',
 'owning_obj',
 'owning_key',
 'related_entity_mnemonic',
 'related_obj',
 'related_key']

In [23]:
list(td_beneficiary_df)

['admin_fk',
 'scheme_fk',
 'scheme_name',
 'ms_pk',
 'mem_status',
 'dep_fk',
 'dep_type',
 'surname',
 'initials',
 'birth_date',
 'gender',
 'race_code',
 'phys1',
 'phys2',
 'phys3',
 'phys4',
 'phys5',
 'phys_code',
 'post1',
 'post2',
 'post3',
 'post4',
 'post5',
 'post_code',
 'start_date',
 'end_date',
 'join_date',
 'resign_date',
 'benefit_date',
 'suspend_date',
 'income_amount',
 'broker_fk',
 'comp_fk',
 'comp_name',
 'load_date',
 'sub_company_fk',
 'subcomp_name',
 'income_cat',
 'resign_code',
 'resign_reason',
 'main_type',
 'relationship',
 'title',
 'main_tel',
 'main_cell',
 'main_work_tel',
 'main_email',
 'comp_post_code',
 'inact_start_1',
 'inact_start_2',
 'inact_end_1',
 'inact_end_2',
 'wait_per_cat',
 'income_proof_date',
 'graduation_date',
 'underwrite_cat',
 'first_name',
 'group_class',
 'group_code',
 'id_num',
 'pref_lang',
 'home_lang',
 'penalty_percentage',
 'orig_joined',
 'main_dep',
 'suspend_type',
 'act_status',
 'nick_name',
 'student_number'

In [24]:
list(tsd_ben_memcom_df)

['comp_fk',
 'comp_name',
 'cre_cont_code',
 'group_class',
 'group_code',
 'cont_person',
 'tel']

In [25]:
list(tsd_dates_df)

['year_number',
 'month_number',
 'day_of_year_number',
 'day_of_month_number',
 'day_of_week_number',
 'week_of_year_number',
 'day_name',
 'month_name',
 'quarter_number',
 'quarter_name',
 'year_quarter_name',
 'weekend_ind',
 'days_in_month_qty',
 'date_pk',
 'day_desc',
 'week_sk',
 'day_date',
 'week_name',
 'week_of_month_number',
 'week_of_month_name',
 'month_sk',
 'quarter_sk',
 'year_sk',
 'year_sort_number',
 'day_of_week_sort_name']

In [26]:
list(legal_raw_2021_df)

['ms_pk',
 'comp_name',
 'comp_fk',
 'user_code',
 'template_label',
 'call_date',
 'dependent_no',
 'problem',
 'sub_category',
 'scheme_fk']

In [27]:
list(councelling_raw_2021_df)

['ms_pk',
 'comp_name',
 'comp_fk',
 'user_code',
 'template_label',
 'call_date',
 'dependent_no',
 'problem',
 'sub_category',
 'scheme_fk']

In [28]:
list(financial_raw_2021_df)

['ms_pk',
 'comp_name',
 'comp_fk',
 'user_code',
 'template_label',
 'call_date',
 'dependent_no',
 'problem',
 'sub_category',
 'scheme_fk']

In [29]:
list(councelling_raw_2021_df)

['ms_pk',
 'comp_name',
 'comp_fk',
 'user_code',
 'template_label',
 'call_date',
 'dependent_no',
 'problem',
 'sub_category',
 'scheme_fk']

In [37]:
councelling_raw_2021_df

,ms_pk,comp_name,comp_fk,user_code,template_label,call_date,dependent_no,problem,sub_category,scheme_fk
0,01690797,Jackpersad & Partners,600404,masetshegob,Offsite Face to Face Counselling Form,2020-05-05,0,Stress,Eposodic Stress,126
1,01736993,AVBOB_Head Office,600509,thabilem,Telephonic Counselling Form,2020-12-17,0,Information,Service Enquiry,126
2,1640427,Capitec_BSC_Western Cape,600206,theoc,Telephonic Counselling Form,2020-03-10,0,Organisational,General Discontent,126
3,1531371,Capitec_BSC_Western Cape,600206,pollyt,Telephonic Counselling Form,2020-06-17,0,Suicide,Contemplated,126
4,511680,Dis-Chem_Direct Courier,101209,puseletsot,Telephonic Counselling Form,2020-05-18,0,Family care,Child Behavioural,108
5,1538479,Reutech Solutions,600043,ludemanc,Telephonic Counselling Form,2020-07-03,0,Information,Service Enquiry,126
6,1572771,Savino Del Bene SA_Johannesburg,600317,puseletsot,Offsite Face to Face Counselling Form,2019-10-31,0,Suicide,Attempted,126
7,1658066,Dis-Chem_Head Office Gauteng,101239,kefilwem,Telephonic Counselling Form,2020-04-06,0,Information,Resources,124
8,1529689,Capitec_Ops Branches_Free State,600209,tshkanim,Telephonic Counselling Form,2019-10-29,0,Relationship,Spouse/Partner,126
9,1612780,Iron Mountain Secure Shredding (Pty) Ltd,101089,nthabelengm,Telephonic Counselling Form,2020-08-04,0,Organisational,Abenteeism/Presenteeism,108


In [33]:
conn_str = (
    "DRIVER={PostgreSQL ODBC Driver(UNICODE)};"
    "DATABASE=kx_bi;"
    "UID=postgres;"
    "PWD=postgres;"
    "SERVER=172.31.244.44;"
    "PORT=5432;"
    )

conn = p.connect(conn_str)

In [34]:
query="""
select distinct
ben.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num 
,ben.dep_fk 
,ben.comp_name
,mipst_dbo.tsf_ccm_template.template_label	
,mipst_dbo.tsf_ccm_template.template_description
,user_code
,replace(substring(call_start_datetime,1,10), '-','') as call_date 
,coalesce(g.comp_name,'none') as group_name	
,dep_type
,ben.comp_fk
,scheme_fk

FROM mipst_dbo.tsf_ccm_template
left join mipst_dbo.tsf_cct_call call ON call.template_obj = mipst_dbo.tsf_ccm_template.template_obj and call.status_key = 'cc_StCctcaComp' 
inner join mipst_dbo.tsd_cc_relationship real ON real.owning_obj::NUMERIC = call.call_obj
and UPPER(owning_entity_mnemonic) = 'CCTCA'
AND   UPPER(related_entity_mnemonic) in ('MEMBER','MEMDEP')
inner join  mipbi_dbo.td_beneficiary ben ON ben.ms_pk = coalesce(nullif(split_part(related_key, '|',1), ''), '0') --and scheme_fk='126'
and dep_fk::int	=coalesce(nullif(split_part(related_key, '|',2), ''), '0')::int
left join mipst_dbo.tsd_ben_memcom g  on  (case when ben.group_code='0' then ben.comp_fk else ben.group_code end)=g.comp_fk
"""

In [52]:
list(cte_df)

['ms_pk',
 'scheme_name',
 'first_name',
 'surname',
 'gender',
 'birth_date',
 'main_email',
 'main_work_tel',
 'id_num',
 'dep_fk',
 'comp_name',
 'template_label',
 'template_description',
 'user_code',
 'call_date',
 'group_name',
 'dep_type',
 'comp_fk',
 'scheme_fk']

In [53]:
set(cte_df.group_name)

{'AFMS Group',
 'AGCO South Africa (Pty) Ltd',
 'ALPLA Trading SA (Pty) Ltd',
 'ANGLO',
 'ARB Group',
 'AT Kearney (Pty) Ltd',
 'AVBOB Mutual Assurance Society',
 'Adams & Adams',
 'Aegis Outsourcing SA (Pty) Ltd',
 'Affmed (Pty) Ltd',
 'Africa Floorcare and Promop (Pty) Ltd',
 'Africa Health Care',
 'African Risk Capacity Insurance Company Limited',
 'AfroCentric Health',
 'Agristar Group',
 'Airwise Air-Conditioning & Ventilation (Pty) Ltd',
 'Amec Foster Wheeler',
 'Amecor Management Services (Pty) Lt',
 'Amecor Pty Ltd',
 'Ampath Trust',
 'Appsure Pty Ltd',
 'Aquila Labour Brokers',
 'Astral Operations Limited',
 'Auram',
 'Avon Justine (Pty) Ltd',
 'BSC Holdings (Pty) Ltd',
 'Bachique Group of Companies',
 'Becton Dickinson Group',
 'Bergman Ross and Partners INC',
 'Beyond Zero',
 'Bidvest Bank Limited',
 'Bidvest Group',
 'Big Five Duty Free',
 'Bombela Operating Company',
 'Boost Business Consulting',
 'Booyco Electronics (Pty) Ltd',
 'Bounty Management Services - Rieses',
 'Bo

In [55]:
capitec_df=cte_df[cte_df.group_name=='Capitec Bank Limited']

In [56]:
capitec_df.to_csv(path_or_buf='C:\\Users\\christo.strydom\\github_repos\\EAP\\data\\capitec_test.csv', sep=',',index=False)

In [35]:
cte_df=pd.read_sql(query,conn)

In [ ]:
limit10_fn(conn,table_name="tsf_ccm_template")

In [5]:
tsf_ccm_template_query="""select *
from mipst_dbo.tsf_ccm_template
limit 10"""
tsf_ccm_template_df=pd.read_sql(tsf_ccm_template_query,conn)

In [7]:
tsf_cct_call_query="""select *
from mipst_dbo.tsf_cct_call
limit 10"""
tsf_cct_call_df=pd.read_sql(tsf_cct_call_query,conn)

In [ ]:
tsf_ccm_template_query="""select *
from mipst_dbo.tsf_ccm_template
limit 10"""
tsf_ccm_template_df=pd.read_sql(tsf_ccm_template_query,conn)

In [ ]:
tsf_ccm_template_query="""select *
from mipst_dbo.tsf_ccm_template
limit 10"""
tsf_ccm_template_df=pd.read_sql(tsf_ccm_template_query,conn)

In [ ]:
tsf_ccm_template_query="""select *
from mipst_dbo.tsf_ccm_template
limit 10"""
tsf_ccm_template_df=pd.read_sql(tsf_ccm_template_query,conn)

In [6]:
tsf_ccm_template_df

,template_obj,status_key,category_key,object_obj,template_code,template_label,template_description,template_field_namelist,template_field_attriblist,last_modified_datetime
0,1001440.0,cc_StCcmtmDis,ma_TmplMemberQry,0.0,ma_TmplMemberReqAuth,Member Authorisation Request,Member Authorisation Request,None,None,2014-08-22 20:46:44:481 + 02:00
1,1001443.0,cc_StCcmtmDis,ma_TmplMemberQry,0.0,ma_TmplMemberChrQry,Member Chronic Query,Member Chronic Query,None,None,2014-08-22 20:46:44:481 + 02:00
2,1001448.0,cc_StCcmtmAct,ma_TmplMemberQry,23417501.0,ma_MemberClaims,Member Claims Query,Member Claims Query,None,None,2017-06-14 14:06:50:939 + 02:00
3,1001464.0,cc_StCcmtmDis,ma_TmplGeneralEny,0.0,ma_GeneralEnquiry,General Query,General Query,None,None,2014-08-22 20:46:44:481 + 02:00
4,1001475.0,cc_StCcmtmDis,ma_TmplMemberQry,0.0,ma_TmplMemberAuthQry,Member Authorisation Query,Member Authorisation Query,None,None,2014-08-22 20:46:44:481 + 02:00
5,1001617.0,cc_StCcmtmDis,ma_TmplServiceProvQry,0.0,ma_ServiceProvAuth,Service Provider Authorisation,Service Provider Authorisation,None,None,2014-08-22 20:46:44:481 + 02:00
6,1004837.0,cc_StCcmtmDis,ma_TmplMemberQry,0.0,ma_TmplMemberGenQry,Member General Enquiry,Member General Enquiry,None,None,2014-08-22 20:46:44:481 + 02:00
7,1288107.0,cc_StCcmtmDis,ma_TmplMarketingQry,0.0,ma_TmplMarketingPaidQry,Paid,Paid Claims,None,None,2014-08-22 20:46:44:481 + 02:00
8,1288132.0,cc_StCcmtmDis,ma_TmplMarketingQry,0.0,ma_TmplMarketingIncorRejectQry,Incorrect Rejection,Incorrect Rejection,None,None,2014-08-22 20:46:44:481 + 02:00
9,1288182.0,cc_StCcmtmDis,ma_TmplMarketingQry,0.0,ma_TmplMarketingAuthReqQry,Authorisation Required,Authorisation Required,None,None,2014-08-22 20:46:44:481 + 02:00


***
Use driver __SQL Server__ and access the __master db__ on MIP __kx-jhb-sql\sql01__ server

In [ ]:
conn_str = (
    r'Driver={SQL Server};'
    r'Server=kx-jhb-sql\sql01;'
    r'Database=master;'
    r'Trusted_Connection=yes;'
    )
cnxn = p.connect(conn_str)

Define the sql script:

In [ ]:
sql_str="""SELECT policyNumber, PolicyJoinDate, PolicyExitDate, ClaimID, Link, PatientSurname, PatientName, PatientIDnr, PatientGender, Product, Hospital, PatientRelationship, ProviderType, AdmissionDate, DischargeDate, ClaimIllness, ClaimProcedure, Surgeon, PracticeNr, ClaimCaptureDate, ClaimProcessDate, ClaimAuthDate, ClaimPayDate, Cftotalclaim, CFTotalCharge, CFTotalPaid, CFTotalMsb, Icdcode, ICDShort, ICDDesription, ChronicIndicator, ICDChapter, MemberName, MemberSurname, DateOfBirth, IDNumber, CellPhoneNumber, MemberEmailAddress, PhysicalAddressLine1, PhysicalAddressLine2, PhysicalAddressLine3, PhysicalPostcode, PostalAddressLine1, PostalAddressLine2, PostalAddressLine3, PostalPostalCode, ClaimStatus, ClaimStatusDesc, RejectionReason, DiagnosisCategory, InsurerCompanyName, IBNR, FamilyBooster, HospitalBooster, CoPayment, DeathBenefit, SubLimit, TarrifShortfalls, GroupType, PayMonth, IncurredMonth, PolicyMonth, MonthsBetween, IBNRClaims, IncurredDateFinal, ProcessDateFinal, PayDateFinal, IncurredToCaptured, CapturedToProcessed, ProcessedToPaid, insert_dt, Broker, comp_code, Company, Other, ClaimNumber, Claim_Code, Claim_Description, MasterCompany, Customer_ID
FROM AnalyticsAndReports.dbo.AgilityClaimsHistory;"""

Connect:

In [ ]:
cursor = cnxn.cursor()
# cursor.execute(sql_str)

Return data in to pandas dataframe:

In [ ]:
AgilityClaimsHistory_df = pd.read_sql(sql_str,cnxn)

In [ ]:
AgilityClaimsHistory_df

In [ ]:
cursor.close()
# cnxn.close()

In [ ]:
list(AgilityClaimsHistory_df)

In [ ]:
sql_query="""SELECT  
case when "mem-num" is null then 'N' else 'Y' end as "Registered Chronic",
c.scheme_fk,
scheme_name, 
ms_fk,
 c.dep_fk as dependent_no,
upper(dep_type) as dep_type,
gender,
b.first_name,
b.surname,
birth_date,
 claim_fk,
 claim_date,
 pr_fk as practice_no,
pr_descr as doctor_name,
 pr_type,
pr_type_descr, 
pr_group_descr,
 tariff_fk, 
claim_type, 
c.claim_code,
description as claim_code_description,
icd10_fk,
icd10_descr,
 three_letter_level as icd10_group_description,
 auth_fk, 
units, 
pmb, 
(Case WHEN c.claim_code = 'K' THEN 'Y' 
		WHEN c.claim_code IN ('C','P') AND c.claim_code = '80' THEN 'Y' 
		WHEN pr_type = '62' AND c.claim_code = '31' THEN 'Y'
		WHEN pr_type IN ('87','88','90') THEN 'Y'
		ELSE 'N' 
	END) as In_Hospital_Indicator,

claimed_amount, 
benefit_amount,
(case when (claimed_amount > 0 AND benefit_amount = 0) and paid_date is not null then 'Rejected' else
case when ((claimed_amount > 0 AND benefit_amount > 0)) and paid_date is not null then 'Paid' else
case when claimed_amount> 0 and benefit_amount > 0  and paid_date is null and assess_date is not null then 'Pending' else 'Unpaid' end end end ) as "ClaimsStatus",
rcvd_date,
assess_date, 
paid_date,
tariff_amount,
 discount,
owes, 
override,
 gen_no,
 suspend_until,
 suspended,
 assessed_datetime,
b.comp_fk,
b.comp_name ,
coalesce(g.comp_name,'none') as group_name,
broker_fk,
broker_descr

	FROM mipbi_dbo.tf_claim_head_curr c
	left join mipst_dbo.tsd_ccdesc cc on c.claim_code=cc.claim_code and scheme_fk=scheme_code::int
	left join mipst_dbo.tsd_icd10 on icd10_fk=icd10_pk 
	left join mipst_dbo.tsd_practice on pr_fk=pr_pk
	left join mipst_dbo.tsd_provider on pr_type_pk=pr_type
	inner join mipbi_dbo.td_beneficiary b on ms_pk=ms_fk and b.scheme_fk=c.scheme_fk and b.dep_fk=c.dep_fk
                 left join mipst_dbo.tsd_ben_memcom g  on  (case when b.group_code='0' then b.comp_fk else b.group_code end)=g.comp_fk
                 left join mipst_dbo.tsd_broker on broker_pk=broker_fk
                 left join (select distinct
"mem-num",
dependant
from
mipst_dbo.tsd_depcond A
inner JOIN
mipst_dbo.tsd_condition_3 B on (A."cond-code" = B.cond_fk)
where
cdl_fk is not null
and "reference-auth-num" <> ''
and cdl_fk <> '' AND cond_descr <> ''
group by
"mem-num",
dependant) a on "mem-num"=ms_fk and
dependant=c.dep_fk
	where claim_date>=20200101 --and ms_fk='026927';"""

In [ ]:
AgilityClaimsHistory_df = pd.read_sql(sql_query,cnxn)

In [ ]:
server = 'kx-jhb-sql\\sql01' 
database = 'AnalyticsAndReports' 
username = 'kaelojhb\\christo.strydom' 
password = 'Twenty23' 
cnxn = p.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
sql_query="""select * from 
(
with cte as (
select distinct
ben.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num 
,ben.dep_fk 
,ben.comp_name
,mipst_dbo.tsf_ccm_template.template_label	
,mipst_dbo.tsf_ccm_template.template_description
,user_code
,replace(substring(call_start_datetime,1,10), '-','') as call_date 
,coalesce(g.comp_name,'none') as group_name	
,dep_type
,ben.comp_fk
,scheme_fk

FROM mipst_dbo.tsf_ccm_template
left join mipst_dbo.tsf_cct_call call ON call.template_obj = mipst_dbo.tsf_ccm_template.template_obj and call.status_key = 'cc_StCctcaComp' 
inner join mipst_dbo.tsd_cc_relationship real ON real.owning_obj::NUMERIC = call.call_obj
and UPPER(owning_entity_mnemonic) = 'CCTCA'
AND   UPPER(related_entity_mnemonic) in ('MEMBER','MEMDEP')
inner join  mipbi_dbo.td_beneficiary ben ON ben.ms_pk = coalesce(nullif(split_part(related_key, '|',1), ''), '0') --and scheme_fk='126'
and dep_fk::int	=coalesce(nullif(split_part(related_key, '|',2), ''), '0')::int
left join mipst_dbo.tsd_ben_memcom g  on  (case when ben.group_code='0' then ben.comp_fk else ben.group_code end)=g.comp_fk
) select cte.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num as identity_number 
,dep_fk as dependent_no
,dep_type
,cte.comp_name as companyname
, group_name
,cte.template_label	
,cte.template_description
,cte.user_code
, cte.call_date::varchar::date as call_date 
, array_to_string(array_agg(distinct (coalesce((c.Problem||'-'||c.sub_category),'') ||','||coalesce(l.Problem ,'')||','||coalesce(f.Problem ,''))),',') as Problem_name
from cte 
INNER JOIN 
mipst_dbo.tsd_dates on mipst_dbo.tsd_dates.date_pk::VARCHAR =call_date 
left join 
mipst_dbo.legal_raw_2021 l on cte.ms_pk=l.ms_pk and cte.dep_fk=l.dependent_no::int and  cte.call_date::varchar::date =l.call_date and cte.template_label=l.template_label and cte.comp_name=l.comp_name 
				  and  cte.scheme_fk=l.scheme_fk and cte.user_code=l.user_code  left join
mipst_dbo.councelling_raw_2021 c on cte.ms_pk=c.ms_pk and cte.dep_fk=c.dependent_no::int and  cte.call_date::varchar::date =c.call_date and cte.template_label=c.template_label and cte.comp_name=c.comp_name 			 
and  cte.scheme_fk=c.scheme_fk and cte.user_code=c.user_code  left join
mipst_dbo.financial_raw_2021 f on cte.ms_pk=f.ms_pk and cte.dep_fk=f.dependent_no::int and  cte.call_date::varchar::date =f.call_date and cte.template_label=f.template_label and cte.comp_name=f.comp_name 			 
and  cte.scheme_fk=f.scheme_fk and cte.user_code=f.user_code  
group by cte.ms_pk
,scheme_name
,first_name
,surname
,gender	
,birth_date
,main_email	
,main_work_tel	
,id_num  
,dep_fk
,dep_type				  
,cte.comp_name 
, group_name
,cte.template_label	
,cte.template_description
,cte.user_code
, cte.call_date::varchar::date 
) a"""

In [ ]:
AgilityClaimsHistory_df = pd.read_sql(sql_query,cnxn)

In [ ]:
cnxn.close()

In [ ]:
conn_str = (
    r'Driver={SQL Server};'
    r'Server=kx-jhb-sql\sql01;'
    r'Database=master;'
    r'Trusted_Connection=yes;'
    )
cnxn = p.connect(conn_str)

In [ ]:
conn_str = (
    r'Driver={SQL Server};'
    r'Server=kx-jhb-sql\sql01;'
    r'Database=master;'
    r'Trusted_Connection=yes;'
    )
cnxn = p.connect(conn_str)

In [ ]:
sql_query="""SELECT TOP 3 * FROM mipst_dbo.tsf_ccm_template"""
tsf_ccm_template_df = pd.read_sql(sql_query,cnxn)

In [ ]:
server = 'kx-jhb-sql\sql01' 
database = 'AnalyticsAndReports' 
username = 'kaelojhb\christo.strydom' 
password = 'Twenty23' 
cnxn = p.connect('DRIVER={SQL Server};SERVER='+server+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
connStr = ('DRIVER={SQL Server Native Client 10.0};Server=196.34.234.252;port=5432;Network Library=DBMSSOCN;Database=kx_bi;uid=kaelouser;pwd=kaelouser;')

In [ ]:
connStr = ('DRIVER={SQL Server};Server=kx-jhb-sql\sql01;Database=sql01;uid=kaelouser;pwd=kaelouser;')

In [ ]:
connStr = ('DRIVER={SQL Server};Server=kx-jhb-sql\sql01;uid=lmbalula;pwd=lmbalula;')

In [ ]:
conn = p.connect(connStr)

In [ ]:
cursor = cnxn.cursor()

In [ ]:
conn_str = (
    "DRIVER={PostgreSQL ODBC Driver(UNICODE)};"
    "DATABASE=kx_bi;"
    "UID=postgres;"
    "PWD=postgres;"
    "SERVER=172.31.244.44;"
    "PORT=5432;"
    )

In [ ]:
conn = p.connect(conn_str)
cursor = conn.cursor()

In [ ]:
AgilityClaimsHistory_df = pd.read_sql(sql_query,conn)

In [ ]:
cnxn = p.connect('DRIVER={PostgreSQL};Server=172.31.244.44;Port=5432;Database=kx_bi;User ID=kaelouser;Password=kaelouser;String Types=Unicode')

In [ ]:
conn = p.connect(conn_str)

In [ ]:
server = 'tcp:myserver.database.windows.net' 
database = 'mydb' 
username = 'myusername' 
password = 'mypassword' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


In [ ]:
p.connect()

In [ ]:
cnxn=p.connect(driver='{SQL Server}',\n",
    "                                server='BISAN\\BISAN', \n",
    "#                                 database=database_name,\n",
    "                                trusted_connection='yes')